In [ ]:
import bivariate_normal_sampler_classes as bns
import numdifftools as nd
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pylab
import statsmodels.api as sm

In [ ]:
# BIVARIATE GAUSSIAN HEATMAPS

# Generate a bunch of covariance matrices
corrs = [0, 0.5, 0.95]
sd_ratios = [1, 0.1, 0.01]
cur_cov_mat = np.zeros((2,2))
cov_mat_list = list()
for i in range(len(corrs)):
    cur_cov_mat[0, 0] = sd_ratios[0]
    cur_cov_mat[1, 1] = 1
    cur_cov_mat[0, 1] = corrs[i] * 1 * np.sqrt(sd_ratios[0])
    cur_cov_mat[1, 0] = corrs[i] * 1 * np.sqrt(sd_ratios[0])
    cov_mat_list.append(cur_cov_mat.copy())
for j in range(len(sd_ratios)):
    cur_cov_mat[0, 0] = sd_ratios[j]
    cur_cov_mat[1, 1] = 1
    cur_cov_mat[0, 1] = corrs[0] * 1 * np.sqrt(sd_ratios[j])
    cur_cov_mat[1, 0] = corrs[0] * 1 * np.sqrt(sd_ratios[j])
    cov_mat_list.append(cur_cov_mat.copy())

   
metrop_samplers = dict()
heatmaps = dict()
for i in range(len(cov_mat_list)):
    metrop_samplers[i] = bns.bivariate_normal_metrop(init = [0,0], sd_sampler = 1, n_samples = 500000, 
                                           covariance_matrix_target = cov_mat_list[i])
    metrop_samplers[i].sd_sampler = np.sqrt(metrop_samplers[i].cov_mat_target[0,0])
    metrop_samplers[i].run_sampler()
    heatmaps[i], xedges, yedges = heatmap_1, xedges, yedges = np.histogram2d(metrop_samplers[i].sample[:, 1], 
                                                                             metrop_samplers[i].sample[:, 0], 
                                                                             bins=(40,40), 
                                                                             range = [[-3,3], [-3, 3]])
# Plot and save heatmaps
for i in range(len(heatmaps)):
    # Plot heatmap
    plt.clf()
    f, ax = plt.subplots()
    ax.set_ylabel('y')
    ax.set_xlabel('x')
    ax.set_ylim([-3, 3])
    ax.imshow(heatmaps[i], extent=[-3, 3, -3, 3], origin = 'lower')
    metrop_samplers[i].plot_ellipse(ax)
    plt.savefig('heatmap_gauss_' + str(i) + '.png')
    plt.show()

In [ ]:
# EXPERIMENT: ACCEPTANCE RATE METROPOLIS (Correlation constant)
# Generate a bunch of covariance matrices
corr = 0
sd_ratios = [1, 0.1, 0.01, 0.001]
cur_cov_mat = np.zeros((2,2))
cov_mat_list = list()
for i in range(len(sd_ratios)):
    cur_cov_mat[0, 0] = sd_ratios[i]
    cur_cov_mat[1, 1] = 1
    cur_cov_mat[0, 1] = corr * 1 * np.sqrt(sd_ratios[i])
    cur_cov_mat[1, 0] = corr * 1 * np.sqrt(sd_ratios[i])
    cov_mat_list.append(cur_cov_mat.copy())
    
# Initialize instance of metropolis sampler
metropolis = bns.bivariate_normal_metrop(init = [0,0], sd_sampler = 1, n_samples = 100000)
acc_rates = list()

for i in range(len(sd_ratios)):
    metropolis.cov_mat_target = cov_mat_list[i]
    metropolis.run_sampler()
    acc_rates.append(metropolis.acceptance_rate)
    metropolis.plot_sample(annotate = True, save = True, alpha = 0.01)
    metropolis.plot_autocorrelation(0, save = True)   
    
# Plot acceptance rates
f, ax = plt.subplots()
ax.plot(sd_ratios, acc_rates)
ax.set_title("Acceptance Rate vs. Var_1 / Var_2")
ax.set_xlabel("SD1 / SD2")
ax.set_ylabel("Acceptance Rate")
ax.xaxis.set_ticks(sd_ratios)
ax.set_xscale('log')
ax.invert_xaxis()
plt.savefig('metrop_acc_vs_vratio.png')
plt.show()

# To avoid deterioration of the acceptance rate we could decrease the standard deviation of the random walk for example
# Lets see if that solves our problem of traversing the sample space while controlling autocorrelation of the chain
metropolis = bns.bivariate_normal_metrop(init = [0,0], sd_sampler = 1, n_samples = 100000)
acc_rates = list()

metropolis.cov_mat_target = cov_mat_list[len(cov_mat_list) - 1]
sampler_sds = [1, 0.001]
acc_rates = list()
for i in range(len(sampler_sds)):
    metropolis.sd_sampler = sampler_sds[i]
    metropolis.run_sampler()
    acc_rates.append(metropolis.acceptance_rate)
    metropolis.plot_autocorrelation(0, save = True)
    
# Plot acceptance rates
f, ax = plt.subplots()
ax.plot(sampler_sds, acc_rates)
ax.set_title("Acceptance Rate")
ax.set_xlabel("SD of random walk")
ax.set_ylabel("Acceptance Rate")
ax.xaxis.set_ticks(sd_ratios)
ax.set_xscale('log')
ax.invert_xaxis()
plt.savefig('metrop_acc_vs_sdsampler.png')
plt.show()

In [ ]:
# Gibbs sampler sample space exploration Experiment: (sd_ratios not that important)

# Generate a bunch of covariance matrices
correlations = [0, 0.5, 0.9, 0.99]
sd_ratios = [1]
cur_cov_mat = np.zeros((2,2))
cov_mat_list = list()
for i in range(len(correlations)):
    cur_cov_mat[0, 0] = 1
    cur_cov_mat[1, 1] = 1
    cur_cov_mat[0, 1] = correlations[i]
    cur_cov_mat[1, 0] = correlations[i] 
    cov_mat_list.append(cur_cov_mat.copy())

# Initialize instance of gibbs sampler
gibbs = bns.bivariate_normal_gibbs(init = [0,0], n_samples = 100)
acc_rates = list()

for i in range(len(correlations)):
    gibbs.cov_mat_target = cov_mat_list[i]
    gibbs.run_sampler()
    #acc_rates.append(gibbs.acceptance_rate)
    gibbs.plot_sample(annotate = True, save = True, trace_sample_path = True, alpha = 1)
    gibbs.plot_autocorrelation(0, save = True)

In [ ]:
# EXPERIMENT: PERIODICITY OF HMC
L_params = [10, 20, 30, 40, 50]
hmc = bns.bivariate_normal_hmc(init = [0,0], n_samples = 100, L = 32, epsilon_range = [0.20])
hmc.trace_hmc_quantities()
hmc.plot_hamiltonian(save = True)
hmc.plot_coordinates(save = True)
hmc.plot_momentum(save = True)
hmc.run_sampler()
hmc.plot_sample(save = True)
hmc.plot_autocorrelation(n_param = 0, save = True)

In [ ]:
# EXPERIMENT: SENSITIVITY TO TUNING
# HMC is sensitive to correct tuning of the L and epsilon parameters

# EXPERIMENT 1: VARY L
L_params = [10, 20, 30, 40, 50]
hmc = bns.bivariate_normal_hmc(init = [0,0], n_samples = 100, L = 20, epsilon_range = [0.1, 0.13])

for i in range(len(L_params)):
    hmc.L = L_params[i]
    hmc.n_samples = 1000
    hmc.run_sampler()
    hmc.plot_sample()
    hmc.plot_autocorrelation(0)
    
# EXPERIMENT 2: VARY epsilon
epsilon_params = [[0.10],
                  [0.15],
                  [0.25],
                  [0.35], 
                  [0.45], 
                  [0.55], 
                  [0.65], 
                  [0.75]]

# Generate covariance matrice
corr = 0.9
sd_ratios = [0.01]
cur_cov_mat = np.zeros((2,2))
cov_mat_list = list()
for i in range(len(sd_ratios)):
    cur_cov_mat[0, 0] = sd_ratios[i]
    cur_cov_mat[1, 1] = 1
    cur_cov_mat[0, 1] = corr * 1 * np.sqrt(sd_ratios[i])
    cur_cov_mat[1, 0] = corr * 1 * np.sqrt(sd_ratios[i])
    cov_mat_list.append(cur_cov_mat.copy())

# Plot stability of Hamiltonian
hmc = bns.bivariate_normal_hmc(init = [0,0], n_samples = 100, L = 20, epsilon_range = [0.1, 0.13], covariance_matrix_target = cov_mat_list[0])
hamiltonian_mat = np.zeros((len(epsilon_params), 2))
for i in range(len(epsilon_params)):
    hmc.epsilon_range = epsilon_params[i]
    hmc.trace_hmc_quantities()
    hamiltonian_mat[i,:] = [min(hmc.trace[0, :]), max(hmc.trace[0, :])]
    
    
plt.clf()
plt.plot(epsilons, abs(hamiltonian_mat[:,0] - hamiltonian_mat[:,1]), color = 'blue')
plt.plot(epsilons, abs(hamiltonian_mat[:,0] - hamiltonian_mat[:,1]),'.', color = 'black')
plt.ylabel('abs(max[H(q,p)] - min[H(q,p)])')
plt.xlabel('$\epsilon$')
plt.title('Instability of Hamiltonian')
plt.savefig('hmc_instability_of_hamiltonian.png')
plt.show()

In [ ]:
# EXPERIMENT: HMC vs. Gibbs 

# Generate covariance matrice
corr = 0.99
sd_ratios = [1]
cur_cov_mat = np.zeros((2,2))
cov_mat_list = list()
for i in range(len(sd_ratios)):
    cur_cov_mat[0, 0] = sd_ratios[i]
    cur_cov_mat[1, 1] = 1
    cur_cov_mat[0, 1] = corr * 1 * np.sqrt(sd_ratios[i])
    cur_cov_mat[1, 0] = corr * 1 * np.sqrt(sd_ratios[i])
    cov_mat_list.append(cur_cov_mat.copy())
    
hmc = bns.bivariate_normal_hmc(init = [0,0], n_samples = 100, L = 30, epsilon_range = [0.05, 0.06])
hmc.cov_mat_target = cov_mat_list[0]
hmc.run_sampler()
hmc.plot_sample(annotate = True, save = True, alpha = 0.2)
hmc.plot_autocorrelation(0, save = True)


gibbs = bns.bivariate_normal_gibbs(init = [0,0], n_samples = 100)
gibbs.cov_mat_target = cov_mat_list[0]
gibbs.run_sampler()
gibbs.plot_sample(annotate = True, save = True, alpha = 0.2)
gibbs.plot_autocorrelation(0, save = True)

In [ ]:
# EXPERIMENT: HMC vs. Metropolis

# Generate covariance matrice
corr = 0
sd_ratios = [0.001]
cur_cov_mat = np.zeros((2,2))
cov_mat_list = list()
for i in range(len(sd_ratios)):
    cur_cov_mat[0, 0] = sd_ratios[i]
    cur_cov_mat[1, 1] = 1
    cur_cov_mat[0, 1] = corr * 1 * np.sqrt(sd_ratios[i])
    cur_cov_mat[1, 0] = corr * 1 * np.sqrt(sd_ratios[i])
    cov_mat_list.append(cur_cov_mat.copy())
    
#hmc = bns.bivariate_normal_hmc(init = [0,0], n_samples = 5000, L = 30, epsilon_range = [0.05, 0.06])
hmc.cov_mat_target = cov_mat_list[0]
hmc.run_sampler()
hmc.plot_sample(annotate = True, save = True, alpha = 0.01)
hmc.plot_autocorrelation(0, save = True)


#metropolis = bns.bivariate_normal_metrop(init = [0,0], sd_sampler = 0.1, n_samples = 5000)
metropolis.cov_mat_target = cov_mat_list[0]
metropolis.run_sampler()
metropolis.plot_sample(annotate = True, save = True, alpha = 0.01)
metropolis.plot_autocorrelation(0, save = True)